In [1]:
# https://www.tensorflow.org/tutorials/layers
import tensorflow as tf
import numpy as np
sess = tf.InteractiveSession()
#from tensorflow.python.framework import ops

In [2]:
shuffle = np.load('data.npy')

In [3]:
from scipy.io import loadmat
data = loadmat("letters_data.mat")
train_x = data['train_x']

# y needs to be one hot encoded
old_y = data['train_y'] - 1
train_y = np.zeros((len(old_y), 26))
for i in range(len(old_y)):
    train_y[i, old_y[i]] = 1
tx = train_x[shuffle][:100000]
ty = train_y[shuffle][:100000]
vx = train_x[shuffle][100000:]
vy = train_y[shuffle][100000:]

In [4]:
def batch_generator(n):
    while True:
        shuffle = np.random.choice(np.arange(100000), replace=False, size=100000)
        for i in range(1000):
            yield tx[i * n: (i+1) * n], ty[i * n: (i+1) * n]

In [5]:
def tune_regularizer(lamb):
    x = tf.placeholder(tf.float32, shape=[None, 784], name='x')
    y_ = tf.placeholder(tf.float32, shape=[None, 26], name='y_')
    input_layer= tf.reshape(x, [-1, 28, 28, 1], name='input')

    reg = tf.contrib.layers.l2_regularizer(lamb)

    # Conv Pool 1
    conv1 = tf.layers.conv2d(
          inputs=input_layer,
          filters=32,
          kernel_size=[5, 5],
          padding="same",
          activation=tf.nn.relu,
          kernel_regularizer=reg,
          name='conv1')
    pool1 = tf.layers.max_pooling2d(
        inputs=conv1,
        pool_size=[2, 2],
        strides=2,
        name='pool1')

    # Conv Pool 2
    conv2 = tf.layers.conv2d(
          inputs=pool1,
          filters=64,
          kernel_size=[5, 5],
          padding="same",
          activation=tf.nn.relu,
          kernel_regularizer=reg,
          name='conv2')
    pool2 = tf.layers.max_pooling2d(
        inputs=conv2,
        pool_size=[2, 2],
        strides=2,
        name='pool2')

    # Dense layer 1
    training = tf.placeholder(tf.bool)
    pool2_flat = tf.reshape(pool2, [-1, 7 * 7 * 64], name='pool2_flat')
    dense1 = tf.layers.dense(
        inputs=pool2_flat,
        units=200,
        activation=tf.nn.relu,
        kernel_regularizer=reg,
        name='dense1')
    dropout1 = tf.layers.dropout(dense1, name='dropout1', rate=0.4, training=training)

    # Dense layer 2
    dense2 = tf.layers.dense(
        inputs=dropout1,
        units=100,
        activation=tf.nn.relu,
        kernel_regularizer=reg,
        name='dense2')
    dropout2 = tf.layers.dropout(dense2, name='dropout2', rate=0.5, training=training)

    # Logits
    logits = tf.layers.dense(inputs=dropout2, units=26, name='logits')

    cross_entropy = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=logits))
    train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)
    correct_prediction = tf.equal(tf.argmax(logits,1), tf.argmax(y_,1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    

    sess.run(tf.global_variables_initializer())
    batch_gen = batch_generator(100)

    print "lambda:", lamb
    for i in range(30000):        
        batch = next(batch_gen)
        if i%10000 == 9999:
            print 'test_accuracy', accuracy.eval(feed_dict={x: vx, y_: vy, training: False})
            print 'train_accuracy', accuracy.eval(feed_dict={x: batch[0], y_: batch[1], training: False})
        train_step.run(feed_dict={x: batch[0], y_: batch[1], training: True})


In [6]:
b = [0.00003, 0.0001, 0.0003, 0.001]
for lamb in b:
    tune_regularizer(lamb)

lambda: 3e-05
test_accuracy 0.926855
train_accuracy 0.93
test_accuracy 0.938226
train_accuracy 0.95
test_accuracy 0.939435
train_accuracy 0.99


ValueError: Variable conv1/kernel already exists, disallowed. Did you mean to set reuse=True in VarScope? Originally defined at:

  File "<ipython-input-5-f48e7112b573>", line 16, in tune_regularizer
    name='conv1')
  File "<ipython-input-6-711b705d2165>", line 3, in <module>
    tune_regularizer(lamb)
  File "/Users/jszhang/env/lib/python2.7/site-packages/IPython/core/interactiveshell.py", line 2882, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)


1024 --> 512 --> 26 94%

In [6]:
# restore
"""
new_saver = tf.train.import_meta_graph('neural-net-model.meta')
new_saver.restore(sess, tf.train.latest_checkpoint('./'))
all_vars = tf.get_collection('vars')
for v in all_vars:
    v_ = sess.run(v)
    print(v_)"""

"\nnew_saver = tf.train.import_meta_graph('neural-net-model.meta')\nnew_saver.restore(sess, tf.train.latest_checkpoint('./'))\nall_vars = tf.get_collection('vars')\nfor v in all_vars:\n    v_ = sess.run(v)\n    print(v_)"

In [8]:
for i in range(100000):        
    batch = next(batch_gen)
    if i%1000 == 999:
        print 'test_accuracy', accuracy.eval(feed_dict={x: vx, y_: vy, training: False})
    if i%100 == 0:
        train_accuracy = accuracy.eval(feed_dict={
            x:batch[0], y_: batch[1], training: False})
        print 'step', i, 'training accuracy', train_accuracy
    train_step.run(feed_dict={x: batch[0], y_: batch[1], training: True})

#print("test accuracy %g"%accuracy.eval(feed_dict={
#    x: vx, y_: vy, training: False}))

step 0 training accuracy 0.04
step 100 training accuracy 0.53
step 200 training accuracy 0.53
step 300 training accuracy 0.73
step 400 training accuracy 0.72
step 500 training accuracy 0.8
step 600 training accuracy 0.87
step 700 training accuracy 0.85
step 800 training accuracy 0.76
step 900 training accuracy 0.91
step 1000 training accuracy 0.91
step 1100 training accuracy 0.88
step 1200 training accuracy 0.85
step 1300 training accuracy 0.85
step 1400 training accuracy 0.87
step 1500 training accuracy 0.89
step 1600 training accuracy 0.91
step 1700 training accuracy 0.96
step 1800 training accuracy 0.85
step 1900 training accuracy 0.94
step 2000 training accuracy 0.93
step 2100 training accuracy 0.94
step 2200 training accuracy 0.91
step 2300 training accuracy 0.88
step 2400 training accuracy 0.85
step 2500 training accuracy 0.91
step 2600 training accuracy 0.91
step 2700 training accuracy 0.95


KeyboardInterrupt: 

In [9]:
accuracy.eval(feed_dict={x: vx, y_: vy, training: False})

0.91991937

In [14]:
# save
variables = [
    x,
    y_,
    input_layer,
    conv1,
    pool1,
    conv2,
    pool2,
    keep_prob,
    pool2_flat,
    dense1,
    dropout1,
    dense2,
    dropout2,
    logits
]
for var in variables:
    tf.add_to_collection('vars', var)
saver = tf.train.Saver()
saver.save(sess, 'neural-net-model')

'neural-net-model'